# Tutorial 3 - MLRun Serving


##### Mlrun serving can take MLRun models or standard model files and produce managed real-time serverless functions based on Nuclio real-time serverless engine, which can be deployed everywhere

##### Nuclio is a high-performance "serverless" framework focused on data, I/O, and compute intensive workloads. More details can be found on https://github.com/nuclio/nuclio 

##### Simple model serving classes can be written in Python or be taken from a set of pre-developed ML/DL classes, the code can handle complex data, feature preparation, binary data (images/video). The serving engine supports the full lifecycle including auto generation of micro-services, APIs, load-balancing, logging, model monitoring, configuration management, etc.



#### Load Project

In [176]:
from os import path, getenv
from mlrun import load_project

project_path = path.abspath('conf')
project = load_project(project_path)

print(f'Project path: {project_path}\nProject name: {project.name}')

Project path: /User/new-tutorials/conf
Project name: getting-started-tutorial-admin


## Writing A Simple Serving Class


##### The class is initialized automatically by the model server , all you need is to implement two mandatory methods:

##### load() - download the model file(s) and load the model into memory, note this can be done synchronously or asynchronously
##### predict() - accept request payload and return prediction/inference results

##### More detailed information on serving classes can be found here : https://github.com/mlrun/mlrun/blob/master/mlrun/serving/README.md


In [177]:
# nuclio: start-code

##### Below is an example of minimal sklearn serving function example:

In [178]:
from cloudpickle import load
import numpy as np
from typing import List
import mlrun

class ClassifierModel(mlrun.serving.V2ModelServer):
    def load(self):
        """load and initialize the model and/or other elements"""
        model_file, extra_data = self.get_model('.pkl')
        self.model = load(open(model_file, 'rb'))

    def predict(self, body: dict) -> List:
        """Generate model predictions from sample."""
        feats = np.asarray(body['inputs'])
        result: np.ndarray = self.model.predict(feats)
        return result.tolist()

In [179]:
# nuclio: end-code

## Deploying the Model Serving Function (Service)

##### In order to provision a serving function we need to create an MLRun function of type serving , this can be done by using the code_to_function() call from a notebook. We can also import an existing serving function/template from the marketplace.

##### The below code converts the ClassifierModel class sbove to a serving function. The name of the class to be used is set using spec.default_class


In [180]:
from mlrun import code_to_function
serving_fn = code_to_function('mlrun-model', kind='serving',image='mlrun/mlrun')
serving_fn.spec.default_class = 'ClassifierModel'

##### Add the model created in previous notebook by the training function  


In [181]:
serving_fn.add_model('my_model',model_path='store://getting-started-tutorial-admin/train-iris-train_iris_model#b2ec6c1ed096494b88318f01de03f508')


In [182]:
serving_fn = serving_fn.apply(mlrun.mount_v3io(remote='projects',mount_path='/v3io/projects'))

serving_fn = project.set_function(serving_fn)


##### Deploy the function - this will build and deploy a Nuclio serving function. Once function is deployed successfully, http endpoint will be available which can get inference requests, call the predict method and return the result

In [183]:
serving_fn.deploy()


> 2020-12-27 15:31:04,170 [info] Starting remote function deploy
2020-12-27 15:31:04  (info) Deploying function
2020-12-27 15:31:04  (info) Building
2020-12-27 15:31:04  (info) Staging files and preparing base images
2020-12-27 15:31:04  (info) Building processor image
2020-12-27 15:31:05  (info) Build complete
2020-12-27 15:31:11  (info) Function deploy complete
> 2020-12-27 15:31:12,110 [info] function deployed, address=default-tenant.app.product-new.iguazio-cd2.com:30845


'http://default-tenant.app.product-new.iguazio-cd2.com:30845'

## Using the new Model Serving Function


##### New endpoint was created for the function and can be accessed via http


In [184]:
function_address = serving_fn.spec.command
print (f'The address for the function is {function_address} \n')

!curl $function_address

The address for the function is http://default-tenant.app.product-new.iguazio-cd2.com:30845 

{"name": "ModelRouter", "version": "v2", "extensions": []}

### Test server by sending data for inference
##### The invoke method enables to programmatically test the function
##### For infer, specify the model name followed by infer =>  /v2/models/{model_name}/infer
##### For complete model service API commands such as list models, get model health and explain see https://github.com/mlrun/mlrun/blob/master/mlrun/serving/README.md#model-server-api 

In [185]:
my_data = '''{"inputs":[[5.1, 3.5, 1.4, 0.2],[7.7, 3.8, 6.7, 2.2]]}'''
serving_fn.invoke('/v2/models/my_model/infer', my_data)


{'id': '01bc5918-be50-4278-b7b7-80749c77baed',
 'model_name': 'my_model',
 'outputs': [0, 2]}

## View Nuclio serving function in UI

##### In the projects screen, select the project and then select "Real-time functions (nuclio)"

<img src="./images/nuclio-deploy.png" alt="Jobs" width="800"/>

##### Save the project

In [186]:
project.save

<bound method MlrunProject.save of <mlrun.projects.project.MlrunProject object at 0x7fd1a3618150>>

## Done!

Congratulation! You've completed tutorial 3 of the Iguazio Data Science Platform.
Go to [Tutorial 4](tutorial-4.ipynb) to learn how to create an automated pipeline for this project